<a href="https://colab.research.google.com/github/VinoJose/Tensorflow/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

## Check the GPU instance

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d85b9cc7-db0c-f90d-74b6-a031914f946b)


## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-07-05 11:18:23--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-05 11:18:23 (70.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-07-05 11:18:30--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-07-05 11:18:30 (92.1 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Shuffle the training dataset
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
 # What does the test dataset look like?
 test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  #print(row)
  _, text, target = row
  print(f"Target:{target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("----\n")

Target:1 (real disaster)
Text:
...//..// whao.. 12000 Nigerian refugees repatriated from Cameroon http://t.co/HuhWPmryWz

----

Target:0 (not real disaster)
Text:
master0fsloths has a crush: http://t.co/SZX6v0bbjF

----

Target:1 (real disaster)
Text:
Aircraft debris confirmed to be from MH370 - Nation | The Star Online http://t.co/heS0bPU60Y

----

Target:0 (not real disaster)
Text:
Wtf? Her biological father is holding her hostage and her adoptive parents haven't even looked for her. Criminal minds got me fucked up.

----

Target:0 (not real disaster)
Text:
'I tried to save Mido Macia': One of the murder accused has testified he tried to save Mido MaciaÛªs life. http://t.co/vxVfAEEY0q

----



### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)


In [13]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

### Converting text to numbers

### Text vectorization (Tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use default text vertorization parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [16]:
# Find the average number of words in the training tweets
round(sum(len(i.split()) for i in train_sentences)/len(train_sentences))

15

In [17]:
# Setup the text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max number of words in our sequence

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# Fit the vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training set and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original sentence:{random_sentence} \n Vectorized sentence:")
text_vectorizer([random_sentence])

Original sentence:Dramatic Video Shows Plane Landing During Violent Storm http://t.co/rJ9gkJKJJn 
 Vectorized sentence:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[2489,   72,  987,  570, 1075,  246,  357,   84,    1,    0,    0,
           0,    0,    0,    0]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [22]:
print(f"The number of words in the vocab: {len(words_in_vocab)}")
print(f"The top 5 common words in the vocab: {top_5_words}")
print(f"The bottom 5 common words in the vocab: {bottom_5_words}")

The number of words in the vocab: 10000
The top 5 common words in the vocab: ['', '[UNK]', 'the', 'a', 'in']
The bottom 5 common words in the vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding layer

In [23]:
 from tensorflow.keras import layers

 embedding = layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length)
 embedding

In [24]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)

print(f"Original sentence: \n{random_sentence} \n\nEmbedded version:")

# Embed the random sentence (turn it into dence vectors of fixed size)
sample_embed = embedding(text_vectorizer(random_sentence))
sample_embed

Original sentence: 
Precious cargo onesie recalled for choking hazard. http://t.co/0PAMznyYuw 

Embedded version:


<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.01674176, -0.0046284 , -0.00150162, ...,  0.04063174,
         0.01394543,  0.03476046],
       [-0.01184066, -0.03222185, -0.00535766, ...,  0.0352743 ,
        -0.00511339, -0.00888925],
       [-0.03686422, -0.04471252, -0.04139058, ...,  0.02404069,
         0.0033635 , -0.00123288],
       ...,
       [ 0.03363908,  0.03314419,  0.00627245, ..., -0.04914627,
         0.00334793, -0.00850556],
       [ 0.03363908,  0.03314419,  0.00627245, ..., -0.04914627,
         0.00334793, -0.00850556],
       [ 0.03363908,  0.03314419,  0.00627245, ..., -0.04914627,
         0.00334793, -0.00850556]], dtype=float32)>

## Modeling a text dataset (running a series of experiments)

Now we know how to turn text data to series of numbers. We can now start a series of model building experiments.

We'll start with a baseline and move on from there.

### Model 0: Getting a baseline

As with all machine learning experiments, let's create a baseline model. We can create baseline model with SKlearn's Naive Bayes using TF-IDF


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())  
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Out baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Out baseline model achieves an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates the model accuracy, precision, recall and f1 score for a binary classifier
  """
  model_accuracy = accuracy_score(y_true, y_pred)

  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average= 'weighted')
  model_results = {'accuracy' : model_accuracy,
                   'precision' : model_precision,
                   'recall' : model_recall,
                   'f1' : model_f1}
  return model_results


In [29]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

 ### Model 1: A simple dense model

In [30]:
 # Create a tensorboard callback 
 from helper_functions import create_tensorboard_callback

 # Create a directory to save tensorboard logs
 SAVE_DIR = "model_logs "

In [31]:
# Build model with functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector 
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [33]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
            y=train_labels,
            epochs=5,
            validation_data=[val_sentences, val_labels],
            callbacks=[create_tensorboard_callback(dir_name= SAVE_DIR,
                                                  experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs /model_1_dense/20220705-111836
Epoch 1/5
215/215 [==============================] - 5s 5ms/step - loss: 0.6109 - accuracy: 0.6981 - val_loss: 0.5361 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4417 - accuracy: 0.8165 - val_loss: 0.4726 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3474 - accuracy: 0.8618 - val_loss: 0.4581 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2853 - accuracy: 0.8915 - val_loss: 0.4707 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2388 - accuracy: 0.9115 - val_loss: 0.4852 - val_accuracy: 0.7874


In [35]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4852 - accuracy: 0.7874


[0.48521968722343445, 0.787401556968689]

In [36]:
# See predictions
model_1_preds_probs= model_1.predict(val_sentences)
model_1_preds_probs[:5]

array([[0.31151465],
       [0.76284385],
       [0.9981407 ],
       [0.12592272],
       [0.09828532]], dtype=float32)

In [37]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_preds_probs))
model_1_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [38]:
# Calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 0.7874015748031497,
 'f1': 0.7839588199365206,
 'precision': 0.7937136229340627,
 'recall': 0.7874015748031497}

In [39]:
# Is our simple Keras model better than our baseline model?
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Recurrent Neural Network

RNNs are useful for sequence data 

### Model 2: LSTM